<a href="https://colab.research.google.com/github/Harooniqbal4879/AgenticAI/blob/main/GitHub_MCP_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GitHub MCP Server - AI-Enabled with GitHub API + Flask


In [ ]:
# 1️. Install Dependencies
!pip install -q flask flask-ngrok openai requests

In [ ]:
# 2️. Import Libraries
import os
import openai
import requests
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

In [ ]:
# 3️. Set API Keys
openai.api_key = "sk-..."  #  Replace with your OpenAI API key
GITHUB_TOKEN = "ghp_..."   #  Replace with your GitHub personal access token

In [ ]:
# 4️. GitHub Utility Function: Search and Fetch Code from Repo
def search_code_in_repo(repo, query, file_ext="py"):
    url = f"https://api.github.com/search/code?q={query}+in:file+repo:{repo}+extension:{file_ext}"
    headers = {"Authorization": f"token {GITHUB_TOKEN}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        items = response.json().get("items", [])
        results = [{"name": item["name"], "path": item["path"], "url": item["html_url"]} for item in items]
        return results
    else:
        return {"error": f"GitHub API error: {response.status_code}"}

In [ ]:
# 5️. Optional: AI Enhancement via OpenAI GPT
def summarize_file_list(file_list):
    prompt = "Summarize these code files:\n" + "\n".join([f"{f['name']}: {f['url']}" for f in file_list])
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

In [ ]:
# 6️. Create MCP Server API
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/mcp/github-code-query", methods=["POST"])
def handle_github_mcp_request():
    data = request.get_json()
    repo = data.get("repo")
    keyword = data.get("query")

    files = search_code_in_repo(repo, keyword)
    summary = summarize_file_list(files) if isinstance(files, list) else files

    return jsonify({
        "repo": repo,
        "query": keyword,
        "results": files,
        "summary": summary
    })

In [ ]:
# Launch the MCP GitHub Server
app.run()
